<a href="https://colab.research.google.com/github/sergiomora03/AdvancedTopicsAnalytics/blob/main/Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Deployment

by [Alejandro Correa Bahnsen](http://www.albahnsen.com/) & [Iván Torroledo](http://www.ivantorroledo.com/)

version 1.5, June 2020

## Part of the class [Advanced Methods in Data Analysis](https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass)


This notebook is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US).

## Agenda:

1. Creating and saving a model
2. Running the model in batch
3. Exposing the model as an API

## Part 1: Phishing Detection

Phishing, by definition, is the act of defrauding an online user in order to obtain personal information by posing as a trustworthy institution or entity. Users usually have a hard time differentiating between legitimate and malicious sites because they are made to look exactly the same. Therefore, there is a need to create better tools to combat attackers.

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/phishing.csv')

In [2]:
data.head()

,url,phishing
0,http://www.subalipack.com/contact/images/sampl...,1
1,http://fasc.maximecapellot-gypsyjazz-ensemble....,1
2,http://theotheragency.com/confirmer/confirmer-...,1
3,http://aaalandscaping.com/components/com_smart...,1
4,http://paypal.com.confirm-key-21107316126168.s...,1


In [3]:
data.tail()

,url,phishing
39995,http://www.diaperswappers.com/forum/member.php...,0
39996,http://posting.bohemian.com/northbay/Tools/Ema...,0
39997,http://www.tripadvisor.jp/Hotel_Review-g303832...,0
39998,http://www.baylor.edu/content/services/downloa...,0
39999,http://www.phinfever.com/forums/viewtopic.php?...,0


In [4]:
data.phishing.value_counts()

phishing
1    20000
0    20000
Name: count, dtype: int64

### Creating features

In [5]:
data.url[data.phishing==1].sample(50, random_state=1).tolist()

['http://dothan.com.co/gold/austspark/index.htm\n',
 'http://78.142.63.63/%7Enetsysco/process/fc1d9c7ea4773b7ff90925c2902cb5f2\n',
 'http://verify95.5gbfree.com/coverme2010/\n',
 'http://www.racom.com/uploads/productscat/bookmark/ii.php?.rand=13vqcr8bp0gud&cbcxt=mai&email=abuse@tradinghouse.ca\n',
 'http://www.cleanenergytci.com/components/update.logon.l3an7lofamerica/2342343234532534546347677898765432876543345687656543876/\n',
 'http://209.148.89.163/-/santander.co.uk/weblegn/AccountLogin.php\n',
 'http://senevi.com/confirmation/\n',
 'http://www.hellenkeller.cl/tmp/new/noticias/Modulo_de_Atualizacao_Bradesco/index2.php?id=PSO1AM04L3Q6PSBNVJ82QUCO0L5GBSY2KM2U9BYUEO14HCRDVZEMTRB3DGJO9HPT4ROC4M8HA8LRJD5FCJ27AD0NTSC3A3VDUJQX6XFG519OED4RW6Y8J8VC19EAAAO5UF21CHGHIP7W4AO1GM8ZU4BUBQ6L2UQVARVM\n',
 'http://internet-sicherheit.co/de/konflikt/src%3Dde/AZ00276ZZ75/we%3Dhs_0_2/sicherheit/konto_verifizieren/verifizierung.php\n',
 'http://alen.co/docs/cleaner\n',
 'http://rattanhouse.co/Atualizacao_

Contain any of the following:
* https
* login
* .php
* .html
* @
* sign
* ?

In [6]:
keywords = ['https', 'login', '.php', '.html', '@', 'sign']

In [7]:
for keyword in keywords:
    data['keyword_' + keyword] = data.url.str.contains(keyword).astype(int)

* Lenght of the url
* Lenght of domain
* is IP?
* Number of .com

In [8]:
data['lenght'] = data.url.str.len() - 2

In [9]:
domain = data.url.str.split('/', expand=True).iloc[:, 2]

In [10]:
data['lenght_domain'] = domain.str.len()

In [11]:
domain.head(12)

0                                    www.subalipack.com
1             fasc.maximecapellot-gypsyjazz-ensemble.nl
2                                    theotheragency.com
3                                    aaalandscaping.com
4     paypal.com.confirm-key-21107316126168.securepp...
5                              lcthomasdeiriarte.edu.co
6                                       livetoshare.org
7                                            www.i-m.co
8                                     manuelfernando.co
9                                www.bladesmithnews.com
10                                      www.rasbaek.com
11                                      199.231.190.160
Name: 2, dtype: object

In [12]:
data['isIP'] = (domain.str.replace('.', '') * 1).str.isnumeric().astype(int)

In [13]:
data['count_com'] = data.url.str.count('com')

In [14]:
data.sample(15, random_state=4)

,url,phishing,keyword_https,keyword_login,keyword_.php,keyword_.html,keyword_@,keyword_sign,lenght,lenght_domain,isIP,count_com
28607,http://pennstatehershey.org/web/ibd/home/event...,0,0,0,0,0,0,0,80,20,0,0
3689,http://guiadesanborja.com/multiprinter/muestra...,1,0,1,1,0,0,0,81,18,0,1
6405,http://paranaibaweb.com/faleconosco/accounting...,1,0,0,0,1,0,0,65,16,0,1
35355,http://courts.delaware.gov/Jury%20Services/Hel...,0,0,0,0,0,0,0,94,19,0,0
16520,http://erpa.co/tmp/getproductrequest.htm\n,1,0,0,0,0,0,0,39,7,0,0
16196,http://pulapulapipoca.com/components/com_media...,1,0,1,1,0,0,0,239,18,0,4
3810,http://www.dag.or.kr/zboard/icon/visa/img/Atua...,1,0,0,0,0,0,0,62,13,0,0
3005,http://www.amazingdressup.com/wp-content/theme...,1,0,0,0,1,0,0,94,22,0,1
9003,http://web.indosuksesfutures.com/content_file/...,1,0,0,0,0,0,0,80,25,0,1
34704,http://www.nutritionaltree.com/subcat.aspx?cid...,0,0,0,0,0,0,0,69,23,0,1


### Create Model

In [15]:
X = data.drop(['url', 'phishing'], axis=1)

In [16]:
y = data.phishing

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [18]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=3)

In [19]:
cross_val_score(clf, X, y, cv=10)

array([0.75   , 0.754  , 0.755  , 0.7545 , 0.74875, 0.767  , 0.75575,
       0.7485 , 0.75825, 0.7525 ])

In [20]:
clf.fit(X, y)

RandomForestClassifier(max_depth=3, n_jobs=-1)

### Save model

In [21]:
import joblib

In [22]:
joblib.dump(clf, 'model_deployment/phishing_clf.pkl', compress=3)

['model_deployment/phishing_clf.pkl']

## Part 2: Model in batch

See m07_model_deployment.py

In [23]:
from model_deployment.m09_model_deployment import predict_proba

In [24]:
predict_proba('http://www.vipturismolondres.com/com.br/?atendimento=Cliente&/LgSgkszm64/B8aNzHa8Aj.php')

0.7709714289649789

## Part 3: API

Flask is considered more Pythonic than Django because Flask web application code is in most cases more explicit. Flask is easy to get started with as a beginner because there is little boilerplate code for getting a simple app up and running.

First we need to install some libraries

```
pip install flask-restplus
```

Load Flask

In [25]:
!pip install flask-restx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.2 MB/s eta 0:00:00


In [26]:
from flask import Flask
from flask_restx import Api, Resource, fields
import joblib

Create api

In [27]:
app = Flask(__name__)

api = Api(
    app,
    version='1.0',
    title='Phishing Prediction API',
    description='Phishing Prediction API')

ns = api.namespace('predict',
     description='Phishing Classifier')

parser = api.parser()

parser.add_argument(
    'URL',
    type=str,
    required=True,
    help='URL to be analyzed',
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})

Load model and create function that predicts an URL

In [28]:
from model_deployment.m09_model_deployment import predict_proba

In [29]:
@ns.route('/')
class PhishingApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()

        urls = args['URL']
        urls = urls.split('%3B')
        print(urls)

        return {
         "result": predict_proba(urls)
        }, 200

Run API

In [30]:
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Check using

* http://localhost:5000/predict/?URL=http://consultoriojuridico.co/pp/www.paypal.com/
